In [ ]:
import cv2
import os
from skimage.metrics import structural_similarity
from werkzeug.utils import secure_filename
from flask import Flask,request,render_template
UPLOAD_FOLDER = 'mysite/static/uploads'
ALLOWED_EXTENSIONS = set(['png', 'jpg', 'jpeg'])
app = Flask(__name__)
app.config['SEND_FILE_MAX_AGE_DEFAULT'] = 0
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
app.secret_key = "secret key"
def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1] in ALLOWED_EXTENSIONS
def make_sketch(img, img2):
    img2=cv2.resize(img2, (img.shape[1], img.shape[0]))
    g1=cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    g2=cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    (score, diff) = structural_similarity(g1, g2, full=True)
    diff = (diff*255).astype("uint8")
    
    _, tresh = cv2.threshold(diff, 40, 255, cv2.THRESH_BINARY_INV)
    
    cont = cv2.findContours(tresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[0]
    
    cont = [c for c in cont if cv2.contourArea(c)>35]
    
    if len(cont):
        for c in cont:
            x,y,w,h = cv2.boundingRect(c)
            cv2.rectangle(img, (x,y), (x+w,y+h), (0,0,255), 4)

    return img
@app.route('/')
def home():
    return render_template('home.html')
@app.route('/sketch',methods=['POST'])
def sketch():
    file = request.files['file']
    file2 = request.files['file2']
    if file and allowed_file(file.filename) and file2 and allowed_file(file2.filename):
        filename = secure_filename(file.filename)
        filename2 = secure_filename(file2.filename)
        file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
        file2.save(os.path.join(app.config['UPLOAD_FOLDER'], filename2))
        img = cv2.imread(UPLOAD_FOLDER+'/'+filename)
        img2 = cv2.imread(UPLOAD_FOLDER+'/'+filename2)
        sketch_img = make_sketch(img, img2)
        sketch_img_name = filename.split('.')[0]+"_sketch.jpg"
        _ = cv2.imwrite(UPLOAD_FOLDER+'/'+sketch_img_name, sketch_img)
        return render_template('home.html',org_img_name=filename2,sketch_img_name=sketch_img_name)
if __name__ == '__main__':
    app.run(debug=True)